In [1]:
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
    
    
dataset = 'D1'

df = pd.read_csv('alltrials.csv', sep=';')
data = df[df['f1']!=0]

testingD1 = data[data['dataset']==dataset]
trainingD1 = data[data['dataset']!=dataset]

y_train = trainingD1[['f1']]
X_train = trainingD1[['clustering', 'lm', 'k', 'threshold', 'InputEntityProfiles', 'NumberOfAttributes', 'NumberOfDistinctValues', 
                'NumberOfNameValuePairs', 'AverageNVPairsPerEntity', 'AverageDistinctValuesPerEntity', 
                'AverageNVpairsPerAttribute', 'AverageDistinctValuesPerAttribute', 'NumberOfMissingNVpairs', 
                'AverageValueLength', 'AverageValueTokens', 'MaxValuesPerEntity']]
X_test = testingD1[['clustering', 'lm', 'k', 'threshold', 'InputEntityProfiles', 'NumberOfAttributes', 'NumberOfDistinctValues', 
                'NumberOfNameValuePairs', 'AverageNVPairsPerEntity', 'AverageDistinctValuesPerEntity', 
                'AverageNVpairsPerAttribute', 'AverageDistinctValuesPerAttribute', 'NumberOfMissingNVpairs', 
                'AverageValueLength', 'AverageValueTokens', 'MaxValuesPerEntity']]
y_test = testingD1[['f1']]


In [5]:
X_train

,clustering,lm,k,threshold,InputEntityProfiles,NumberOfAttributes,NumberOfDistinctValues,NumberOfNameValuePairs,AverageNVPairsPerEntity,AverageDistinctValuesPerEntity,AverageNVpairsPerAttribute,AverageDistinctValuesPerAttribute,NumberOfMissingNVpairs,AverageValueLength,AverageValueTokens,MaxValuesPerEntity
4050,ConnectedComponentsClustering,sdistilroberta,23,0.557313,2152,3,4383,4876,2.27,2.04,1625.33,1461.0,1580.0,197.28,39.85,3
4051,UniqueMappingClustering,st5,1,0.050000,2152,3,4383,4876,2.27,2.04,1625.33,1461.0,1580.0,197.28,39.85,3
4054,UniqueMappingClustering,sminilm,25,0.725000,2152,3,4383,4876,2.27,2.04,1625.33,1461.0,1580.0,197.28,39.85,3
4055,ConnectedComponentsClustering,sdistilroberta,23,0.557313,2152,3,4383,4876,2.27,2.04,1625.33,1461.0,1580.0,197.28,39.85,3
4056,UniqueMappingClustering,st5,1,0.050000,2152,3,4383,4876,2.27,2.04,1625.33,1461.0,1580.0,197.28,39.85,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41028,KiralyMSMApproximateClustering,st5,82,0.202816,50797,10,424346,971445,19.12,8.35,97144.50,42434.6,65400000.0,283.02,62.72,1306
41029,KiralyMSMApproximateClustering,smpnet,77,0.092164,50797,10,424346,971445,19.12,8.35,97144.50,42434.6,65400000.0,283.02,62.72,1306
41030,UniqueMappingClustering,smpnet,3,0.461966,50797,10,424346,971445,19.12,8.35,97144.50,42434.6,65400000.0,283.02,62.72,1306
41032,BestMatchClustering,st5,79,0.531726,50797,10,424346,971445,19.12,8.35,97144.50,42434.6,65400000.0,283.02,62.72,1306


In [20]:
X_test

array([[-1.68157581, -1.82983911, -0.89090926, ..., -0.51760405,
        -0.49370974,  1.73724002],
       [-0.8582244 ,  0.78513315, -0.89090926, ...,  1.9319787 ,
        -0.49370974, -0.5756257 ],
       [-1.68157581, -1.82983911, -0.89090926, ..., -0.51760405,
        -0.49370974,  1.73724002],
       ...,
       [-1.61296319, -0.23386951, -0.89090926, ..., -0.51760405,
         2.02548163, -0.5756257 ],
       [ 0.99431627,  0.03638231, -0.89090926, ..., -0.51760405,
        -0.49370974,  1.73724002],
       [-1.2699001 , -1.1600535 , -0.89090926, ..., -0.51760405,
        -0.49370974, -0.5756257 ]])

In [2]:
scaler = StandardScaler()
X_train = scaler.fit_transform(pd.get_dummies(X_train))
X_test = scaler.transform(pd.get_dummies(X_test))


In [3]:
import autosklearn.regression as autosklearn

In [9]:
# Initialize Auto-sklearn for regression
automl = autosklearn.AutoSklearnRegressor(
    time_left_for_this_task=60*60,  # Total time for the AutoML process
    per_run_time_limit=300,       # Time for each model
    memory_limit=6144, 
    # verbosity=2,
    ensemble_size=1               # Use single best model
)

# Fit the model
automl.fit(X_train, y_train, dataset_name='trials_optuna')

# Predict using the best model
y_pred = automl.predict(X_test)

# Evaluate the predictions
print("R2 Score:", r2_score(y_test, y_pred))
print("Mean Absolute Error:", mean_absolute_error(y_test, y_pred))
print("Mean Squared Error:", mean_squared_error(y_test, y_pred))

# Display the details of the best model
print(automl.show_models())

R2 Score: -0.7631323075200227
Mean Absolute Error: 12.116659890603897
Mean Squared Error: 253.8808916293008
{7: {'model_id': 7, 'rank': 1, 'cost': 0.0005645414762424084, 'ensemble_weight': 1.0, 'data_preprocessor': <autosklearn.pipeline.components.data_preprocessing.DataPreprocessorChoice object at 0x7f2232633160>, 'feature_preprocessor': <autosklearn.pipeline.components.feature_preprocessing.FeaturePreprocessorChoice object at 0x7f2232633ca0>, 'regressor': <autosklearn.pipeline.components.regression.RegressorChoice object at 0x7f217cb1cd00>, 'sklearn_regressor': ExtraTreesRegressor(bootstrap=True, max_features=0.9615263480351033,
                    n_estimators=512, n_jobs=1, random_state=1,
                    warm_start=True)}}


In [8]:
from pprint import pprint

pprint(automl.show_models(), indent=4)

{   2: {   'cost': 0.0007271268546206189,
           'data_preprocessor': <autosklearn.pipeline.components.data_preprocessing.DataPreprocessorChoice object at 0x7f22312eb550>,
           'ensemble_weight': 1.0,
           'feature_preprocessor': <autosklearn.pipeline.components.feature_preprocessing.FeaturePreprocessorChoice object at 0x7f22312b4b80>,
           'model_id': 2,
           'rank': 1,
           'regressor': <autosklearn.pipeline.components.regression.RegressorChoice object at 0x7f22312b4e20>,
           'sklearn_regressor': RandomForestRegressor(max_features=1.0, n_estimators=512, n_jobs=1,
                      random_state=1, warm_start=True)}}
